Começamos importando as bibliotecas necessárias. Estamos utilizando requests para comunicação com a API da CVM e zipfile junto com io para manipulação de arquivos em memória.

Definimos também as constantes do projeto. Segundo as instruções do desafio, devemos buscar dados do Portal de Dados Abertos da CVM. Organizamos a saída na pasta data/raw para manter a estrutura do projeto limpa.

In [1]:
import os
import requests
import zipfile
import io
import pandas as pd
from datetime import datetime, timedelta

# Definição dos diretórios
OUTPUT_DIR = '../data/raw'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# URL base oficial da CVM para o Informe Diário
BASE_URL = "https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS"

Aqui criamos uma função robusta para o processo de ETL (Extract, Transform, Load).

Decisão Técnica: Ao invés de baixar o arquivo .zip para o disco e depois descompactar (o que geraria lixo e uso desnecessário de I/O), optamos por baixar o fluxo de bytes diretamente na memória RAM usando io.BytesIO. Isso torna o processamento mais rápido e eficiente. Se o download falhar (ex: erro de conexão), o código captura a exceção para não interromper todo o fluxo abruptamente.

In [2]:
def download_e_extrair_cvm(ano, mes):
    """
    Baixa o arquivo ZIP da CVM para um mês específico e extrai o CSV na pasta de destino.
    """
    nome_arquivo = f"inf_diario_fi_{ano}{mes:02d}.zip"
    url = f"{BASE_URL}/{nome_arquivo}"

    print(f"🔄 Baixando: {nome_arquivo}...")

    try:
        response = requests.get(url, stream=True)
        response.raise_for_status() # Garante que o link existe

        # Manipulação em memória para extração direta
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            z.extractall(OUTPUT_DIR)

        print(f"✅ Sucesso: {nome_arquivo} extraído.")

    except Exception as e:
        print(f"❌ Erro ao baixar {nome_arquivo}: {e}")

O desafio sugere um período mínimo de 12 meses, mas indica que 24 meses é desejável. Para garantir robustez na validação do modelo (backtesting), optamos por baixar os últimos 2 anos de dados.

Este bloco percorre o calendário mês a mês, chamando nossa função de extração.

In [3]:
# Janela de tempo: Últimos 24 meses
data_final = datetime.now()
data_inicial = data_final - timedelta(days=365 * 2)

print(f"🚀 Iniciando download de {data_inicial.strftime('%m/%Y')} até {data_final.strftime('%m/%Y')}...")

data_atual = data_inicial
while data_atual <= data_final:
    download_e_extrair_cvm(data_atual.year, data_atual.month)

    # Avança para o dia 1 do próximo mês
    # (Soma 32 dias e volta para o dia 1 para garantir a virada de mês)
    data_atual = (data_atual.replace(day=1) + timedelta(days=32)).replace(day=1)

🚀 Iniciando download de 12/2023 até 12/2025...
🔄 Baixando: inf_diario_fi_202312.zip...
✅ Sucesso: inf_diario_fi_202312.zip extraído.
🔄 Baixando: inf_diario_fi_202401.zip...
✅ Sucesso: inf_diario_fi_202401.zip extraído.
🔄 Baixando: inf_diario_fi_202402.zip...
✅ Sucesso: inf_diario_fi_202402.zip extraído.
🔄 Baixando: inf_diario_fi_202403.zip...
✅ Sucesso: inf_diario_fi_202403.zip extraído.
🔄 Baixando: inf_diario_fi_202404.zip...
✅ Sucesso: inf_diario_fi_202404.zip extraído.
🔄 Baixando: inf_diario_fi_202405.zip...
✅ Sucesso: inf_diario_fi_202405.zip extraído.
🔄 Baixando: inf_diario_fi_202406.zip...
✅ Sucesso: inf_diario_fi_202406.zip extraído.
🔄 Baixando: inf_diario_fi_202407.zip...
✅ Sucesso: inf_diario_fi_202407.zip extraído.
🔄 Baixando: inf_diario_fi_202408.zip...
✅ Sucesso: inf_diario_fi_202408.zip extraído.
🔄 Baixando: inf_diario_fi_202409.zip...
✅ Sucesso: inf_diario_fi_202409.zip extraído.
🔄 Baixando: inf_diario_fi_202410.zip...
✅ Sucesso: inf_diario_fi_202410.zip extraído.
🔄 Baixa

Além dos dados de cota e PL (Informe Diário), o desafio exige o uso de atributos do fundo  para filtrar nossa classe de ativos.

Baixamos o arquivo cad_fi.csv (Cadastro de Fundos), que contém o CNPJ, a Denominação Social e, crucialmente, a Classe CVM. Isso será usado na próxima etapa para filtrar apenas os fundos de Ações com mais de 50 cotistas, conforme o escopo.

In [4]:
print("\n🔄 Baixando Cadastro de Fundos (Metadados)...")
url_cad = "https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv"

try:
    response = requests.get(url_cad)
    with open(f"{OUTPUT_DIR}/cad_fi.csv", 'wb') as f:
        f.write(response.content)
    print("✅ Cadastro (cad_fi.csv) salvo com sucesso!")
except Exception as e:
    print(f"❌ Falha no download do cadastro: {e}")


🔄 Baixando Cadastro de Fundos (Metadados)...
✅ Cadastro (cad_fi.csv) salvo com sucesso!
